필요한 함수를 다른 파일에서 가져와 사용

### 감정 분석 자연어 처리
1. data 폴더 안의 ratings_train.txt 파일을 로드한다.
2. 상위 500개 데이터만 추출한다.
    - case 2) 25,000번째부터 30,000번째의 데이터를 이용해보기
3. 리뷰 데이터와 감정 데이터로 나눠준다.
4. 리뷰 데이터를 토큰화한다. (Komoran 이용)
5. 토큰화한 데이터를 Word2Vec에 학습시킨다.
    - window = 3
    - epochs = 10
    - min_count = 5
    - sg = 1
    - seed = 42
6. 벡터화한다. (Word2Vec, 단위 벡터의 평균 이용)
7. 분류 모델 (SVC, Logistic)
8. Train, Test 를 이용하여 2개의 모델 중 성능이 더 높은 모델을 찾는다.
9. 단위 벡터의 평균의 성능과 단위벡터 + 중요도 평균의 성능 차이를 확인한다.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from konlpy.tag import Komoran
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# 데이터 파일 로드
try:
    df = pd.read_csv('../data/ratings_train.txt', sep='\t')
except FileNotFoundError:
    print("파일 경로를 확인해주세요.'../data/ratings_train.txt'")
except Exception as e:
    # ParserError 방지 등을 위해 sep='\t'가 필수적
    print(f"데이터 로드 오류: {e}")

In [4]:
df.head(1)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [ ]:
# 상위 500개의 데이터만 추출
# df2 = df.head(500)
df2 = df.loc[:500]

In [7]:
df2['label'].value_counts()

label
1    259
0    242
Name: count, dtype: int64

In [8]:
# 'document' 컬럼 결측치 제거 및 타입 변환 (Java Exception 방지)
df['document'] = df['document'].fillna('').astype(str)

**각 데이터가 후속 처리 과정에서 요구하는 자료형에 따라 tolist(), values 사용**
- ```X_text``` (리뷰 텍스트)
    - 텍스트 처리
    - 자연어 처리(NLP), 특히 토큰화(```tokenize_komoran```)와 Word2Vec 학습을 위해 사용되는 데이터
    - Word2Vec이나 대부분의 텍스트 처리 함수는 입력 데이터로 파이썬 리스트(List) 형태를 선호.
    </br>특히 ```Word2Vec(sentences=...)``` 매개변수는 문장들의 리스트를 요구.
    - ```.tolist()```의 역할: Pandas Series를 순수한 Python 리스트로 변환하여, NLP 라이브러리와의 호환성을 높이고 처리 과정에서 발생할 수 있는 Pandas 인덱싱 관련 오류를 방지.
- ```Y_label``` (감정 라벨)
    - 수치 학습
    - 최종적으로 분류 모델(```SVC```, ```LogisticRegression```)의 종속 변수로 사용되는 데이터
    - Scikit-learn 모델은 종속 변수로 NumPy 배열 형태를 선호.
    - ```.values```의 역할: Pandas Series에서 NumPy 배열만 효율적으로 추출하여 반환합니다. 이는 불필요한 Pandas 인덱스 정보를 제외하고 순수한 숫자 배열만 모델 학습에 사용되도록 하여 처리 효율성을 높입니다

In [ ]:
# 토큰화
# Komoran 객체 생성
try:
    komoran = Komoran()
except Exception as e:
    print("Komoran 초기화 오류. Java 환경 또는 PyKomoran 설치를 확인하세요: ", e)

def build_tokenize():
    try:
        # 라이브러리 로드 -> 라이브러리가 존재하면 코드들 실행 
        from konlpy.tag import Komoran  # 'K'moran 대문자임을 주의
        komoran = Komoran()
        allow_pos = ['NNP', 'NNG', 'VV', 'VA', 'SL', 'MAG']
        def tokenize(text):
            tokens = []
            for word, pos in komoran.pos(text):
                if pos in allow_pos:
                    tokens.append(word)
            return tokens
        # tokenize 함수를 결과로 되돌려준다.
        # 함수 자체를 되돌려줄 때는 함수명() 이 아닌 함수명만 되돌려줘야 함을 주의
        return tokenize
    except Exception as e:
        print("Komoran 사용 불가: ", e)
        return lambda x: x.split()

In [ ]:
# build_tokenize 함수 호출
tokenize = build_tokenize()

In [ ]:
# 리뷰 데이터와 감정 데이터로 분할
reviews = df['document'].values
Y_label = df['label'].values

In [ ]:
X_tokens = [ tokenize(review) for review in reviews]

In [ ]:
# Word2Vec을 이용하여 학습(Skip-gram 방식)
w2v = Word2Vec(
    sentences= X_tokens, 
    window = 5, 
    min_count= 2, 
    sg = 1, 
    epochs= 100, 
    seed = 42
)

wv = w2v.wv

In [ ]:
def sent_embed_mean(tokens):
    vecs = []
    for word in tokens:
        if word in wv.index_to_key:
            vecs.append(wv[word])
    result = np.mean(vecs, axis=0) if vecs else np.zeros(wv.vector_size)
    return result

In [ ]:
# 단어별 중요도 
tfidf_vec = TfidfVectorizer(
    tokenizer= tokenize, 
    lowercase= False
).fit(reviews)
idf = dict(
    zip(
        # get_feature_names_out() -> Tfidf에서 사용된 단어들의 목록
        tfidf_vec.get_feature_names_out(), 
        # idf_ : 중요도
        tfidf_vec.idf_
    )
)
# 단어 별 단위 벡터의 평균과 idf을 곱한다. 
def sent_embed_tfidf(tokens):
    vecs = []
    weight = []
    for word in tokens:
        # tokens에 각각의 단어가 Word2Vec과 TF-IDF에 존재한다면
        if word in wv.key_to_index and word in idf:
            # vecs -> 단위벡터와 중요도를 곱한 값을 vecs 추가
            vecs.append(wv[word] * idf[word])
            # weight -> 중요도 데이터를 추가 
            weight.append(idf[word])
    # vecs의 데이터가 존재하지 않는다면 -> tokens 안에 단어는 존재하지만 Word2Vec이나
    # TD-IDF에 단어가 존재하지 않을때
    if not vecs:
        # 희소 행렬 되돌려준다. 0행렬
        result = np.zeros(wv.vector_size)
    else:
        result = np.sum(vecs, axis=0) / ( np.sum(weight) + 1e-9 )
    return result

In [ ]:
X_embed = [ sent_embed_mean(token) for token in X_tokens ]
X_embed

In [ ]:
X_embed2 = [ sent_embed_tfidf(token) for token in X_tokens ]
X_embed2

In [ ]:
# 모델 2개 객체 생성 
svc = SVC(random_state= 42)
logi = LogisticRegression(random_state=42)

In [ ]:
def run_model(X, Y, model, test_size = 0.2):
    # X는 독립 변수
    # Y는 종속 변수
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size= test_size, random_state=42, stratify=Y
    )
    # 모델에 학습
    model.fit(X_train, Y_train)
    # 학습된 모델에 예측 값
    y_pred = model.predict(X_test)
    print("정확도 : ", round(
        accuracy_score(y_pred, Y_test), 4
    ))
    print('분류 레포드 : ')
    print(classification_report(y_pred, Y_test))

In [ ]:
run_model(X_embed, Y, svc)

In [ ]:
run_model(X_embed, Y, logi)

In [ ]:
run_model(X_embed2, Y, svc)

In [ ]:
run_model(X_embed2, Y, logi)

df에서 하위 10개 데이터를 이용하여 예측

In [ ]:
# 학습된 모델에 예측의 값을 반환하는 함수 
# 세번째 매개변수(vec_type)를 생성 -> 기본값은 'mean'
# 'tfidf' 입력이 들어온다면 벡터화 작업은 w2v + ifidf 융합한 벡터화 
def predict_sentence_list(sentences, model, vec_type = 'mean'):
    # sentences : 문장들의 리스트 
    # 문장들을 토큰화 -> 임베딩 
    X_test = []
    for sent in sentences:
        # token() 함수를 호출하여 토큰화 
        tokens = tokenize(sent)
        # 토큰화 된 문장을 sent_enbed_mean 함수에 입력하여 호출 ( 단위 백터의 평균 )
        if vec_type == 'mean':
            vec = sent_embed_mean(tokens)
        elif vec_type == 'tfidf':
            vec = sent_embed_tfidf(tokens)
        X_test.append(vec)
    
    preds = model.predict(X_test)
    result = []
    for sent, pred in zip(sentences, preds):
        label = "긍정" if pred == 1 else "부정"
        result.append([sent, label])
    return result

In [ ]:
# 모델 학습 -> 예측
X_test = df['document'].tail(10).values
run_model(X_embed, Y, svc)

predict_sentence_list(X_test, svc, vec_type='mean')

In [ ]:

run_model(X_embed2, Y, logi)
predict_sentence_list(X_test, logi, vec_type='tfidf')